# The Honoré (1992) Trimmed LAD Estimator for Fixed Effects Tobit

## Learning Objectives

- Understand why standard fixed effects Tobit is inconsistent (incidental parameters problem)
- Learn Honoré's semiparametric approach using pairwise differencing
- Estimate fixed effects censored models using the Trimmed LAD estimator in PanelBox
- Compare Honoré's estimator with Random Effects Tobit and understand trade-offs
- Recognize practical considerations: computational cost, convergence, and limitations

## Duration
90-120 minutes

## Prerequisites
- Tobit model fundamentals (censored regression)
- Fixed effects vs. random effects panel models
- Maximum likelihood estimation concepts
- Familiarity with LAD (Least Absolute Deviations) / median regression

## Dataset
Consumer durables spending panel: household-level spending on durable goods
- N = 200 households, T = 5 periods
- Outcome: `spending` (censored at 0 -- many households report zero spending)
- Predictors: `income`, `wealth`, `household_size`, `homeowner`, `urban`, `credit_score`

## Level
**Advanced** -- This notebook covers an experimental, computationally intensive estimator.
It is intended for researchers who need fixed effects estimation in censored models
and understand the theoretical limitations of standard approaches.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Statistical libraries
from scipy import stats
import statsmodels.api as sm

# PanelBox imports
from panelbox.models.censored import PooledTobit, RandomEffectsTobit, HonoreTrimmedEstimator

# Visualization configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Set random seed for reproducibility
np.random.seed(42)

# Define paths (relative to notebook location in examples/censored/notebooks/)
BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = BASE_DIR / 'outputs'
FIGURES_DIR = OUTPUT_DIR / 'figures'
TABLES_DIR = OUTPUT_DIR / 'tables'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print('Setup complete!')

---

## 1. The Fixed Effects Problem in Censored Models

### The Standard Fixed Effects Tobit Model

Consider a panel data Tobit model with individual fixed effects:

$$y_{it}^* = \mathbf{X}_{it}'\boldsymbol{\beta} + \alpha_i + \varepsilon_{it}$$

$$y_{it} = \max(0, y_{it}^*)$$

where:
- $y_{it}^*$ is the latent (uncensored) outcome
- $y_{it}$ is the observed outcome, left-censored at 0
- $\alpha_i$ is the individual fixed effect (unobserved heterogeneity)
- $\varepsilon_{it} \sim \text{i.i.d.}(0, \sigma^2)$

### Why Not Just Estimate Fixed Effects MLE?

In linear models, the within-transformation (demeaning) eliminates $\alpha_i$ and produces consistent estimates of $\boldsymbol{\beta}$ even with fixed $T$. However, in nonlinear models like the Tobit, this approach fails.

### The Incidental Parameters Problem (Neyman and Scott, 1948)

If we attempt to estimate $\alpha_i$ for each entity $i$ alongside $\boldsymbol{\beta}$, we face the **incidental parameters problem**:

1. The number of parameters to estimate grows with $N$ (one $\alpha_i$ per entity).
2. Each $\alpha_i$ is estimated using only $T$ observations.
3. When $T$ is fixed (small), the MLEs of $\alpha_i$ are **inconsistent**.
4. This inconsistency **contaminates** the MLE of $\boldsymbol{\beta}$ through the likelihood function.

In contrast to linear models, the Tobit likelihood is **not separable** in $(\boldsymbol{\beta}, \alpha_i)$, so:

$$\hat{\boldsymbol{\beta}}_{\text{FE-MLE}} \nrightarrow \boldsymbol{\beta}_0 \quad \text{as } N \to \infty \text{ with } T \text{ fixed}$$

This bias can be substantial in practice -- simulations show that fixed effects MLE Tobit can overestimate coefficients by 50-100% or more when $T$ is small.

### Available Solutions

| Approach | Assumption on $\alpha_i$ | Consistent? | Distributional Assumptions |
|----------|--------------------------|-------------|----------------------------|
| Pooled Tobit | Ignores $\alpha_i$ | Only if $\alpha_i = 0$ | Normal errors |
| Random Effects Tobit | $\alpha_i \perp \mathbf{X}_{it}$ | Under RE assumption | Normal errors + RE |
| FE MLE Tobit | None on $\alpha_i$ | **No** (incidental params) | Normal errors |
| **Honoré Trimmed LAD** | **None on** $\alpha_i$ | **Yes** (semiparametric) | **Minimal** |

The Honoré (1992) estimator is the only approach that allows **arbitrary correlation** between $\alpha_i$ and $\mathbf{X}_{it}$ while maintaining consistency.

In [ ]:
# Simulation: Demonstrate the incidental parameters bias
# We generate data from a known DGP and show that FE MLE is biased

np.random.seed(42)

# True parameters
beta_true = 1.5
sigma_true = 1.0

# Simulation settings
N_sim = 100       # number of entities
T_values = [3, 5, 10, 25, 50]  # varying T to show bias shrinks with T
n_reps = 200      # Monte Carlo replications

results_sim = []

for T_sim in T_values:
    beta_estimates = []
    
    for rep in range(n_reps):
        # Generate fixed effects correlated with X
        alpha_i = np.random.normal(0, 2, size=N_sim)
        
        # Generate panel data
        x_list, y_list = [], []
        for i in range(N_sim):
            x_it = np.random.normal(alpha_i[i] * 0.5, 1, size=T_sim)  # correlated with alpha
            eps_it = np.random.normal(0, sigma_true, size=T_sim)
            y_star = beta_true * x_it + alpha_i[i] + eps_it
            y_it = np.maximum(0, y_star)  # left-censoring at 0
            x_list.append(x_it)
            y_list.append(y_it)
        
        x_all = np.concatenate(x_list)
        y_all = np.concatenate(y_list)
        
        # Naive approach: OLS on uncensored observations (ignoring FE)
        uncensored = y_all > 0
        if uncensored.sum() > 2:
            X_unc = sm.add_constant(x_all[uncensored])
            beta_ols = np.linalg.lstsq(X_unc, y_all[uncensored], rcond=None)[0][1]
            beta_estimates.append(beta_ols)
    
    mean_est = np.mean(beta_estimates)
    bias = mean_est - beta_true
    results_sim.append({
        'T': T_sim,
        'Mean Estimate': mean_est,
        'True Value': beta_true,
        'Bias': bias,
        'Relative Bias (%)': 100 * bias / beta_true
    })

sim_df = pd.DataFrame(results_sim)
print('Incidental Parameters Bias Demonstration')
print('(Naive OLS on uncensored obs, ignoring FE -- correlated alpha_i)')
print('=' * 70)
print(f'True beta = {beta_true}')
print(f'N = {N_sim}, Monte Carlo reps = {n_reps}')
print()
display(sim_df)

In [ ]:
# Visualize bias as a function of T
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: estimated vs true beta
axes[0].plot(sim_df['T'], sim_df['Mean Estimate'], 'o-', markersize=8,
             linewidth=2, label='Mean Estimate')
axes[0].axhline(y=beta_true, color='red', linestyle='--', linewidth=2,
                label=f'True $\\beta$ = {beta_true}')
axes[0].set_xlabel('Number of Time Periods (T)', fontsize=12)
axes[0].set_ylabel('Estimated $\\beta$', fontsize=12)
axes[0].set_title('Bias from Ignoring Fixed Effects\nin Censored Models', fontsize=13)
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Right panel: relative bias
colors = ['#D55E00' if b > 5 else '#009E73' for b in sim_df['Relative Bias (%)'].abs()]
axes[1].bar(range(len(sim_df)), sim_df['Relative Bias (%)'], color=colors, alpha=0.7,
            edgecolor='black')
axes[1].set_xticks(range(len(sim_df)))
axes[1].set_xticklabels([f'T={t}' for t in sim_df['T']])
axes[1].axhline(y=0, color='black', linewidth=1)
axes[1].set_xlabel('Number of Time Periods (T)', fontsize=12)
axes[1].set_ylabel('Relative Bias (%)', fontsize=12)
axes[1].set_title('Relative Bias in $\\beta$ Estimation\n(Ignoring Correlated Fixed Effects)', fontsize=13)
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'incidental_parameters_bias.png', dpi=300, bbox_inches='tight')
plt.show()

print('Key insight: With correlated fixed effects, naive estimation is biased.')
print('The incidental parameters problem makes standard FE MLE inconsistent for fixed T.')

*Figure: Left panel shows estimated beta converging toward the true value as T increases, but remaining biased when T is small. Right panel displays the relative bias in percentage terms. Orange bars indicate substantial bias (above 5%), green bars indicate smaller bias. The key takeaway is that ignoring correlated fixed effects in censored models produces biased estimates, especially with short panels.*

---

## 2. Honoré's Semiparametric Approach

### The Core Idea: Pairwise Differencing

Honoré (1992) proposed an elegant solution: instead of trying to estimate the fixed effects, **eliminate them through pairwise differencing**.

For any two time periods $t$ and $s$ of entity $i$:

$$y_{it}^* - y_{is}^* = (\mathbf{X}_{it} - \mathbf{X}_{is})'\boldsymbol{\beta} + (\varepsilon_{it} - \varepsilon_{is})$$

The fixed effect $\alpha_i$ cancels out! However, we observe $y_{it} = \max(0, y_{it}^*)$, not $y_{it}^*$, so we cannot simply difference the observed outcomes.

### The Trimming Strategy

Honoré's key insight is that the **conditional median** of the differenced data can be used to form a valid estimating equation, provided we **trim** pairs where both observations are censored.

The intuition is:
- If both $y_{it}$ and $y_{is}$ are censored at 0, the pair $(y_{it} - y_{is}) = 0$ provides no information about $\boldsymbol{\beta}$.
- If at least one observation is uncensored, the difference contains information about $\boldsymbol{\beta}$.

### The Trimmed LAD Objective Function

The estimator minimizes a trimmed Least Absolute Deviations (LAD) objective:

$$\hat{\boldsymbol{\beta}}_{\text{TLAD}} = \arg\min_{\boldsymbol{\beta}} \sum_{i=1}^{N} \sum_{t < s} w_{its} \left| (y_{it} - y_{is}) - (\mathbf{X}_{it} - \mathbf{X}_{is})'\boldsymbol{\beta} \right|$$

where the trimming weights are:

$$w_{its} = \begin{cases} 1 & \text{if not both } y_{it} \text{ and } y_{is} \text{ are censored} \\ 0 & \text{if both } y_{it} \text{ and } y_{is} \text{ are censored at } 0 \end{cases}$$

### Why LAD Instead of OLS?

The use of absolute deviations (LAD/median regression) rather than squared deviations (OLS) is crucial:

1. **Robustness**: LAD is robust to asymmetry in the error distribution caused by censoring.
2. **Consistency**: Under mild regularity conditions, the trimmed LAD estimator is consistent for $\boldsymbol{\beta}$ as $N \to \infty$ with $T$ fixed.
3. **Minimal assumptions**: No distributional assumptions on $\varepsilon_{it}$ beyond median zero, and no assumptions on $\alpha_i$ whatsoever.

### Properties of the Estimator

- **Consistent**: $\hat{\boldsymbol{\beta}} \to \boldsymbol{\beta}_0$ as $N \to \infty$ with $T$ fixed.
- **Semiparametric**: No distributional assumptions on $\alpha_i$ or $\varepsilon_{it}$.
- **Asymptotically normal**: Under regularity conditions, $\sqrt{N}(\hat{\boldsymbol{\beta}} - \boldsymbol{\beta}_0) \xrightarrow{d} N(0, V)$.
- **Trade-off**: No standard errors are computed analytically (the asymptotic variance is complex); bootstrap is recommended.

---

## 3. Loading and Exploring the Data

We use a panel dataset on consumer durables spending. Households may report zero spending (left-censored at 0), and individual heterogeneity (e.g., permanent preferences, liquidity constraints) creates fixed effects that may be correlated with covariates.

In [ ]:
# Load consumer durables panel data
df = pd.read_csv(DATA_DIR / 'consumer_durables_panel.csv')

print('Dataset shape:', df.shape)
print(f'Unique households: {df["id"].nunique()}')
print(f'Time periods: {df["time"].nunique()}')
print()
print('First few rows:')
display(df.head(10))

print('\nVariable types:')
print(df.dtypes)

In [ ]:
# Summary statistics
print('Summary Statistics')
print('=' * 70)
display(df.describe())

# Censoring information
n_censored = (df['spending'] == 0).sum()
n_total = len(df)
pct_censored = 100 * n_censored / n_total

print(f'\nCensoring Summary:')
print(f'  Total observations:  {n_total}')
print(f'  Censored (spending=0): {n_censored} ({pct_censored:.1f}%)')
print(f'  Uncensored (spending>0): {n_total - n_censored} ({100 - pct_censored:.1f}%)')
print(f'  Mean spending (all):  {df["spending"].mean():.2f}')
print(f'  Mean spending (uncensored): {df.loc[df["spending"] > 0, "spending"].mean():.2f}')

---

## 4. Exploratory Analysis: Individual Heterogeneity and Censoring Patterns

Before estimating models, we need to understand:
1. How much individual heterogeneity exists (justifying fixed effects)
2. The censoring patterns across individuals and time

In [ ]:
# Analyze censoring patterns by individual
censoring_by_id = df.groupby('id').agg(
    n_censored=('spending', lambda x: (x == 0).sum()),
    n_uncensored=('spending', lambda x: (x > 0).sum()),
    mean_spending=('spending', 'mean'),
    mean_income=('income', 'mean')
).reset_index()

censoring_by_id['pct_censored'] = 100 * censoring_by_id['n_censored'] / df.groupby('id').size().values

# Distribution of censoring frequency across households
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Left: histogram of censoring frequency per household
axes[0].hist(censoring_by_id['n_censored'], bins=range(0, 7), alpha=0.7,
             edgecolor='black', color='steelblue')
axes[0].set_xlabel('Number of Censored Periods (out of 5)')
axes[0].set_ylabel('Number of Households')
axes[0].set_title('Censoring Frequency per Household')
axes[0].grid(alpha=0.3)

# Middle: spending distribution by censoring status
uncensored_spending = df.loc[df['spending'] > 0, 'spending']
axes[1].hist(uncensored_spending, bins=30, alpha=0.7, edgecolor='black',
             color='#009E73')
axes[1].set_xlabel('Spending (Uncensored Only)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Positive Spending')
axes[1].grid(alpha=0.3)

# Right: mean spending vs mean income colored by censoring rate
scatter = axes[2].scatter(
    censoring_by_id['mean_income'],
    censoring_by_id['mean_spending'],
    c=censoring_by_id['pct_censored'],
    cmap='RdYlGn_r', s=40, alpha=0.7, edgecolor='gray', linewidth=0.5
)
plt.colorbar(scatter, ax=axes[2], label='% Censored')
axes[2].set_xlabel('Mean Income')
axes[2].set_ylabel('Mean Spending')
axes[2].set_title('Income vs Spending\n(Color = Censoring Rate)')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'honore_censoring_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

*Figure: Left panel shows the distribution of censoring frequency per household -- most households have 0 to 2 censored periods, but some are censored in all 5 periods. Middle panel shows the right-skewed distribution of positive spending amounts. Right panel plots mean income against mean spending for each household, colored by their censoring rate; households with higher income tend to have higher spending and lower censoring rates, suggesting correlation between covariates and individual heterogeneity.*

In [ ]:
# Evidence for individual heterogeneity: within vs between variation
# If alpha_i matters, between-individual variation should be large

overall_var = df['spending'].var()
between_var = df.groupby('id')['spending'].mean().var()
within_var = df.groupby('id')['spending'].apply(lambda x: x.var()).mean()

print('Variance Decomposition of Spending')
print('=' * 50)
print(f'Overall variance:   {overall_var:.2f}')
print(f'Between variance:   {between_var:.2f} ({100 * between_var / overall_var:.1f}%)')
print(f'Within variance:    {within_var:.2f} ({100 * within_var / overall_var:.1f}%)')
print()

if between_var / overall_var > 0.3:
    print('Substantial between-individual variation detected.')
    print('This suggests individual heterogeneity (alpha_i) is important.')
    print('Fixed effects estimation is warranted.')
else:
    print('Limited between-individual variation.')
    print('Individual heterogeneity may be less important.')

# Check correlation between individual means of X and spending
# If alpha_i is correlated with X, RE will be biased
individual_means = df.groupby('id')[['spending', 'income', 'wealth', 'credit_score']].mean()
print('\nCorrelation Between Individual Means (Evidence for Correlated Effects):')
print('-' * 60)
for var in ['income', 'wealth', 'credit_score']:
    corr = individual_means['spending'].corr(individual_means[var])
    print(f'  Corr(mean spending, mean {var}): {corr:.3f}')

In [ ]:
# Visualize individual heterogeneity: spaghetti plot of spending trajectories
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Spaghetti plot for a subsample of individuals
sample_ids = df['id'].unique()[:30]  # first 30 households
for hh_id in sample_ids:
    hh_data = df[df['id'] == hh_id]
    axes[0].plot(hh_data['time'], hh_data['spending'], alpha=0.4, linewidth=1)

axes[0].set_xlabel('Time Period')
axes[0].set_ylabel('Spending')
axes[0].set_title('Individual Spending Trajectories\n(30 households)')
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Censoring point')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Right: Boxplot of spending by time period
df.boxplot(column='spending', by='time', ax=axes[1])
axes[1].set_xlabel('Time Period')
axes[1].set_ylabel('Spending')
axes[1].set_title('Spending Distribution by Period')
plt.suptitle('')  # Remove automatic title from boxplot
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'honore_individual_heterogeneity.png', dpi=300, bbox_inches='tight')
plt.show()

*Figure: Left panel shows spending trajectories for 30 households across 5 time periods, illustrating substantial individual heterogeneity -- some households consistently spend more while others frequently report zero spending. The red dashed line marks the censoring point at zero. Right panel shows boxplots of spending by time period, revealing the overall distribution and the mass point at zero (censoring).*

---

## 5. The Trimmed LAD Estimator: Mathematical Formulation

### Step-by-Step Construction

**Step 1: Pairwise Differencing**

For each entity $i$ and each pair of time periods $(t, s)$ with $t < s$, compute:

$$\Delta y_{its} = y_{it} - y_{is}$$
$$\Delta \mathbf{X}_{its} = \mathbf{X}_{it} - \mathbf{X}_{is}$$

**Step 2: Apply Trimming**

Define the trimming indicator:

$$w_{its} = \mathbb{1}\{\text{not both } y_{it} = 0 \text{ and } y_{is} = 0\}$$

Pairs where both observations are censored provide no information about $\boldsymbol{\beta}$ and are discarded.

**Step 3: Minimize the Trimmed LAD Objective**

$$Q_N(\boldsymbol{\beta}) = \sum_{i=1}^{N} \sum_{t < s} w_{its} \left| \Delta y_{its} - \Delta \mathbf{X}_{its}'\boldsymbol{\beta} \right|$$

This is a non-smooth optimization problem (the absolute value function is not differentiable at zero), which requires specialized optimization techniques.

### Computational Considerations

1. **Number of pairs**: For $T$ periods per entity, there are $\binom{T}{2} = T(T-1)/2$ pairs. With $T=5$, that is 10 pairs per entity.
2. **Total observations**: $N \times \binom{T}{2}$ pairwise differences. For $N=200$ and $T=5$: 2,000 pairs.
3. **Non-smooth objective**: The LAD objective requires subgradient-based methods (e.g., L-BFGS-B with smoothed approximation).
4. **No closed-form SE**: Standard errors require bootstrap or kernel-based estimation of the asymptotic variance.

In [ ]:
# Illustrate the pairwise differencing concept with a small example
# Take one household and show all pairwise differences

example_id = df['id'].unique()[0]
example_data = df[df['id'] == example_id].sort_values('time')

print(f'Example: Household {example_id}')
print('=' * 60)
display(example_data[['time', 'spending', 'income', 'wealth']].reset_index(drop=True))

print('\nAll Pairwise Differences:')
print('-' * 60)

T_hh = len(example_data)
spending = example_data['spending'].values
income = example_data['income'].values
times = example_data['time'].values

pairs_data = []
for t in range(T_hh):
    for s in range(t + 1, T_hh):
        delta_y = spending[t] - spending[s]
        delta_x = income[t] - income[s]
        both_censored = (spending[t] == 0) and (spending[s] == 0)
        trim = 'TRIMMED' if both_censored else 'Kept'
        
        pairs_data.append({
            'Pair (t, s)': f'({times[t]}, {times[s]})',
            'y_t': spending[t],
            'y_s': spending[s],
            'dy': round(delta_y, 2),
            'dx (income)': round(delta_x, 2),
            'Status': trim
        })

pairs_df = pd.DataFrame(pairs_data)
display(pairs_df)

n_kept = (pairs_df['Status'] == 'Kept').sum()
n_trimmed = (pairs_df['Status'] == 'TRIMMED').sum()
print(f'\nKept: {n_kept} pairs, Trimmed: {n_trimmed} pairs')
print('Pairs where both observations are censored at 0 provide no information about beta.')

---

## 6. Estimation with PanelBox: HonoreTrimmedEstimator

PanelBox provides the `HonoreTrimmedEstimator` class for estimating the Honoré (1992) Trimmed LAD model.

**Important notes:**
- This estimator is **experimental** and computationally intensive.
- It issues an `ExperimentalWarning` upon instantiation.
- Do **not** include a constant column in `exog` -- the fixed effects absorb the intercept.
- For larger datasets, consider using a subsample for initial exploration.

### API Overview

```python
model = HonoreTrimmedEstimator(
    endog=y,              # 1D array of dependent variable
    exog=X,               # 2D array of covariates (NO constant)
    groups=groups,        # Entity identifiers
    time=time,            # Time identifiers
    censoring_point=0.0   # Left-censoring threshold
)

result = model.fit()      # Returns HonoreResults dataclass
```

In [ ]:
# Use a SMALLER subsample for the Honore estimator
# This estimator is computationally expensive due to pairwise differencing
# We select n=50 households (250 observations with T=5)

np.random.seed(42)
all_ids = df['id'].unique()
subsample_ids = np.random.choice(all_ids, size=50, replace=False)
df_sub = df[df['id'].isin(subsample_ids)].copy().reset_index(drop=True)

print(f'Subsample size: {len(df_sub)} observations')
print(f'Households: {df_sub["id"].nunique()}')
print(f'Time periods per household: {df_sub.groupby("id").size().unique()}')
print(f'Censored observations: {(df_sub["spending"] == 0).sum()} ({100 * (df_sub["spending"] == 0).mean():.1f}%)')

# Prepare arrays for estimation
# IMPORTANT: Do NOT include a constant -- FE absorbs the intercept
y_honore = df_sub['spending'].values
X_honore = df_sub[['income', 'wealth', 'household_size', 'credit_score']].values
groups_honore = df_sub['id'].values
time_honore = df_sub['time'].values

var_names_honore = ['income', 'wealth', 'household_size', 'credit_score']

print(f'\nCovariates: {var_names_honore}')
print(f'X shape: {X_honore.shape}')
print(f'Number of pairwise differences per entity: {5 * 4 // 2} = C(5,2)')
print(f'Total pairwise differences: {50 * 10} (before trimming)')

In [ ]:
# Estimate the Honore Trimmed LAD model
# Wrap in try/except since this is experimental

print('Estimating Honor\u00e9 Trimmed LAD Estimator...')
print('=' * 60)
print('NOTE: This estimator is experimental. An ExperimentalWarning will be shown.')
print()

try:
    # Re-enable warnings temporarily to show the ExperimentalWarning
    with warnings.catch_warnings():
        warnings.simplefilter('always')
        
        honore_model = HonoreTrimmedEstimator(
            endog=y_honore,
            exog=X_honore,
            groups=groups_honore,
            time=time_honore,
            censoring_point=0.0
        )
    
    # Fit the model
    honore_result = honore_model.fit(verbose=True)
    
    print('\n' + '=' * 60)
    print('Estimation Complete!')
    print(f'Converged: {honore_result.converged}')
    print(f'Iterations: {honore_result.n_iter}')
    print(f'Observations used: {honore_result.n_obs}')
    print(f'Entities: {honore_result.n_entities}')
    print(f'Trimmed pairs: {honore_result.n_trimmed}')
    
    honore_success = True

except Exception as e:
    print(f'\nEstimation failed with error: {e}')
    print('This can happen with small samples or degenerate data.')
    print('Try increasing the sample size or adjusting starting values.')
    honore_success = False

In [ ]:
# Display the full model summary
if honore_success:
    print(honore_model.summary())
    
    print('\n\nCoefficient Interpretation:')
    print('-' * 50)
    for i, name in enumerate(var_names_honore):
        coef = honore_result.params[i]
        print(f'  {name:>20s}: {coef:>10.4f}')
    
    print('\nNote: Standard errors are NOT available for the Honore estimator.')
    print('Bootstrap is recommended for inference (see Practical Considerations).')
else:
    print('Skipping summary -- estimation did not succeed.')

---

## 7. Comparing Random Effects Tobit vs. Honoré Trimmed LAD

To put the Honoré estimates in context, let us estimate a Random Effects Tobit model on the same subsample and compare the results.

### When to Use Each Estimator

| Criterion | Random Effects Tobit | Honoré Trimmed LAD |
|-----------|---------------------|---------------------|
| Assumption on $\alpha_i$ | $\alpha_i \perp \mathbf{X}_{it}$ | **None** (can be correlated) |
| Distributional assumptions | Normal $\alpha_i$, normal $\varepsilon_{it}$ | **Minimal** (median zero errors) |
| Standard errors | Available (from Hessian) | **Not available** (use bootstrap) |
| Efficiency | More efficient if assumptions hold | Less efficient |
| Robustness | Biased if RE assumptions fail | **Robust** to correlated effects |
| Computational cost | Moderate (quadrature) | **High** (pairwise differencing) |
| Intercept | Estimated | Not estimated (absorbed by FE) |

In [ ]:
# Estimate Random Effects Tobit on the same subsample for comparison
# RE Tobit requires a constant in X

X_re = sm.add_constant(X_honore)  # Add constant for RE model

print('Estimating Random Effects Tobit...')
print('=' * 60)

try:
    re_tobit = RandomEffectsTobit(
        endog=y_honore,
        exog=X_re,
        groups=groups_honore,
        time=time_honore,
        censoring_point=0.0
    )
    
    re_result = re_tobit.fit()
    print(re_tobit.summary())
    re_success = True

except Exception as e:
    print(f'RE Tobit estimation failed: {e}')
    re_success = False

In [ ]:
# Build comparison table
if honore_success and re_success:
    # RE Tobit coefficients (skip constant, sigma_eps, sigma_alpha)
    re_beta = re_tobit.beta[1:]  # skip constant
    re_se = re_tobit.bse[1:re_tobit.n_features]  # skip constant, exclude sigma params
    
    comparison_data = []
    for i, name in enumerate(var_names_honore):
        comparison_data.append({
            'Variable': name,
            'Honore TLAD': honore_result.params[i],
            'RE Tobit': re_beta[i],
            'RE Tobit SE': re_se[i] if i < len(re_se) else np.nan,
            'Difference': honore_result.params[i] - re_beta[i],
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df['Rel. Diff (%)'] = 100 * comparison_df['Difference'] / comparison_df['RE Tobit'].abs()
    
    print('Coefficient Comparison: Honore vs RE Tobit')
    print('=' * 80)
    display(comparison_df)
    
    print('\nInterpretation:')
    print('-' * 60)
    print('  - Large differences suggest correlated individual effects.')
    print('  - If alpha_i is correlated with X, RE Tobit is inconsistent.')
    print('  - Honore is consistent regardless of the correlation structure.')
    print('  - Honore does not provide standard errors (bootstrap needed).')
    
    # Save comparison table
    comparison_df.to_csv(TABLES_DIR / 'honore_vs_re_comparison.csv', index=False)

else:
    print('Cannot create comparison -- one or both estimations failed.')

In [ ]:
# Visualize the comparison
if honore_success and re_success:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: Coefficient comparison bar chart
    x_pos = np.arange(len(var_names_honore))
    width = 0.35
    
    bars1 = axes[0].bar(x_pos - width/2, honore_result.params, width,
                         label='Honor\u00e9 TLAD', color='steelblue', alpha=0.8,
                         edgecolor='black')
    bars2 = axes[0].bar(x_pos + width/2, re_beta, width,
                         label='RE Tobit', color='#D55E00', alpha=0.8,
                         edgecolor='black')
    
    # Add RE Tobit error bars
    axes[0].errorbar(x_pos + width/2, re_beta,
                      yerr=1.96 * re_se[:len(var_names_honore)],
                      fmt='none', color='black', capsize=3, capthick=1.5)
    
    axes[0].set_xticks(x_pos)
    axes[0].set_xticklabels(var_names_honore, rotation=15, ha='right')
    axes[0].set_ylabel('Coefficient Estimate')
    axes[0].set_title('Coefficient Comparison:\nHonor\u00e9 TLAD vs RE Tobit')
    axes[0].legend()
    axes[0].axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
    axes[0].grid(alpha=0.3, axis='y')
    
    # Right: Relative difference
    rel_diff = comparison_df['Rel. Diff (%)'].values
    colors_bar = ['#D55E00' if abs(d) > 20 else '#009E73' for d in rel_diff]
    axes[1].bar(x_pos, rel_diff, color=colors_bar, alpha=0.7, edgecolor='black')
    axes[1].set_xticks(x_pos)
    axes[1].set_xticklabels(var_names_honore, rotation=15, ha='right')
    axes[1].set_ylabel('Relative Difference (%)')
    axes[1].set_title('Relative Difference Between Estimators\n(Honor\u00e9 - RE Tobit) / |RE Tobit|')
    axes[1].axhline(y=0, color='black', linewidth=1)
    axes[1].grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'honore_vs_re_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

else:
    print('Cannot create comparison plot -- estimation(s) failed.')

*Figure: Left panel compares coefficient estimates from the Honor\u00e9 Trimmed LAD estimator (blue) and the Random Effects Tobit (orange) with 95% confidence intervals for the RE model. Right panel shows the relative difference between the two estimators for each variable. Orange bars indicate substantial differences (above 20%), which may suggest that the RE assumption of uncorrelated individual effects is violated.*

In [ ]:
# Additional comparison: Pooled Tobit (ignoring all panel structure)
if honore_success:
    print('Estimating Pooled Tobit for three-way comparison...')
    print('=' * 60)
    
    try:
        pooled_tobit = PooledTobit(
            endog=y_honore,
            exog=X_re,  # with constant
            groups=groups_honore,
            censoring_point=0.0
        )
        pooled_result = pooled_tobit.fit()
        
        pooled_beta = pooled_tobit.beta[1:]  # skip constant
        
        print('\nThree-Way Coefficient Comparison')
        print('=' * 70)
        
        three_way = pd.DataFrame({
            'Variable': var_names_honore,
            'Pooled Tobit': pooled_beta[:len(var_names_honore)],
            'RE Tobit': re_beta[:len(var_names_honore)] if re_success else [np.nan] * len(var_names_honore),
            'Honore TLAD': honore_result.params
        })
        display(three_way)
        
        print('\nKey differences indicate the importance of accounting for')
        print('individual heterogeneity and its correlation with covariates.')
        
    except Exception as e:
        print(f'Pooled Tobit estimation failed: {e}')

---

## 8. Practical Considerations

### 8.1 Computational Cost

The Honoré estimator's main computational bottleneck is the **pairwise differencing** step:

- For $N$ entities and $T$ periods: $N \times \binom{T}{2}$ pairwise differences.
- With $N = 200$ and $T = 5$: $200 \times 10 = 2{,}000$ pairs.
- With $N = 1{,}000$ and $T = 10$: $1{,}000 \times 45 = 45{,}000$ pairs.
- With $N = 5{,}000$ and $T = 20$: $5{,}000 \times 190 = 950{,}000$ pairs.

The computational cost scales as $O(N \cdot T^2)$ for constructing differences and $O(N \cdot T^2 \cdot K \cdot I)$ for optimization, where $K$ is the number of covariates and $I$ is the number of iterations.

In [ ]:
# Demonstrate computational scaling
import time

# Test with different subsample sizes
subsample_sizes = [20, 30, 40, 50]
timing_results = []

for n_hh in subsample_sizes:
    np.random.seed(42)
    sub_ids = np.random.choice(all_ids, size=n_hh, replace=False)
    df_timing = df[df['id'].isin(sub_ids)].copy().reset_index(drop=True)
    
    y_t = df_timing['spending'].values
    X_t = df_timing[['income', 'wealth', 'household_size', 'credit_score']].values
    g_t = df_timing['id'].values
    t_t = df_timing['time'].values
    
    start_time = time.time()
    
    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            model_t = HonoreTrimmedEstimator(
                endog=y_t, exog=X_t, groups=g_t, time=t_t, censoring_point=0.0
            )
            result_t = model_t.fit(verbose=False)
        
        elapsed = time.time() - start_time
        n_pairs = n_hh * 10  # C(5,2) = 10 pairs per entity
        
        timing_results.append({
            'N households': n_hh,
            'N observations': n_hh * 5,
            'N pairs': n_pairs,
            'Time (s)': round(elapsed, 3),
            'Converged': result_t.converged
        })
    
    except Exception as e:
        elapsed = time.time() - start_time
        timing_results.append({
            'N households': n_hh,
            'N observations': n_hh * 5,
            'N pairs': n_hh * 10,
            'Time (s)': round(elapsed, 3),
            'Converged': f'Error: {str(e)[:30]}'
        })

timing_df = pd.DataFrame(timing_results)
print('Computational Scaling of Honore Estimator')
print('=' * 60)
display(timing_df)

In [ ]:
# Visualize computational scaling
if len(timing_df) > 1:
    fig, ax = plt.subplots(figsize=(10, 5))
    
    ax.plot(timing_df['N households'], timing_df['Time (s)'], 'o-',
            markersize=8, linewidth=2, color='steelblue')
    
    ax.set_xlabel('Number of Households (N)', fontsize=12)
    ax.set_ylabel('Computation Time (seconds)', fontsize=12)
    ax.set_title('Computational Cost of Honor\u00e9 Estimator\n(T=5 periods per household)', fontsize=13)
    ax.grid(alpha=0.3)
    
    # Add annotation
    ax.annotate('Cost scales as O(N * T^2)',
                xy=(timing_df['N households'].iloc[-1], timing_df['Time (s)'].iloc[-1]),
                xytext=(-80, 20), textcoords='offset points',
                fontsize=11, fontstyle='italic',
                arrowprops=dict(arrowstyle='->', color='gray'))
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'honore_computational_scaling.png', dpi=300, bbox_inches='tight')
    plt.show()

*Figure: Computation time of the Honor\u00e9 Trimmed LAD estimator as a function of the number of households, with T=5 periods each. The cost grows with N as expected, following the pairwise differencing complexity O(N * T^2).*

### 8.2 Convergence Issues

Since the Trimmed LAD objective is **non-smooth** (based on absolute values), convergence can be challenging:

1. **Non-differentiability**: The absolute value function has a kink at zero. PanelBox uses a subgradient approximation.
2. **Local minima**: The objective may have local minima, especially with few observations.
3. **Starting values**: Good starting values (e.g., from OLS on differenced data) improve convergence.
4. **Optimization method**: L-BFGS-B is used by default; for difficult cases, consider Nelder-Mead (gradient-free).

### 8.3 Inference Without Standard Errors

The Honoré estimator in PanelBox does **not** provide analytical standard errors. The asymptotic variance of the estimator involves a kernel density estimate of the error distribution, which is complex to implement reliably.

**Recommended approaches for inference:**

1. **Bootstrap**: Resample entities (not individual observations) and re-estimate. The entity-level bootstrap preserves the within-entity correlation structure.
2. **Comparison with RE Tobit**: If both estimators give similar results, the RE assumption may hold, and the RE Tobit standard errors can be used as an approximation.
3. **Sensitivity analysis**: Estimate on multiple subsamples to assess stability.

In [ ]:
# Bootstrap standard errors for the Honore estimator
# Entity-level bootstrap: resample households (not individual obs)

if honore_success:
    n_bootstrap = 50  # Keep small for demonstration; use 200+ in practice
    bootstrap_params = []
    
    print(f'Running entity-level bootstrap ({n_bootstrap} replications)...')
    print('This may take a few minutes.')
    print()
    
    unique_ids_sub = df_sub['id'].unique()
    n_entities_sub = len(unique_ids_sub)
    
    for b in range(n_bootstrap):
        # Resample entities with replacement
        boot_ids = np.random.choice(unique_ids_sub, size=n_entities_sub, replace=True)
        
        # Build bootstrap sample (renumber IDs to handle duplicates)
        boot_dfs = []
        for new_id, orig_id in enumerate(boot_ids):
            entity_data = df_sub[df_sub['id'] == orig_id].copy()
            entity_data['boot_id'] = new_id
            boot_dfs.append(entity_data)
        
        boot_df = pd.concat(boot_dfs, ignore_index=True)
        
        # Estimate on bootstrap sample
        try:
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                boot_model = HonoreTrimmedEstimator(
                    endog=boot_df['spending'].values,
                    exog=boot_df[['income', 'wealth', 'household_size', 'credit_score']].values,
                    groups=boot_df['boot_id'].values,
                    time=boot_df['time'].values,
                    censoring_point=0.0
                )
                boot_result = boot_model.fit(verbose=False)
                
                if boot_result.converged:
                    bootstrap_params.append(boot_result.params)
        except Exception:
            pass  # Skip failed bootstrap replications
        
        if (b + 1) % 10 == 0:
            print(f'  Completed {b + 1}/{n_bootstrap} replications '
                  f'({len(bootstrap_params)} successful)')
    
    # Compute bootstrap standard errors
    if len(bootstrap_params) >= 10:
        boot_params_array = np.array(bootstrap_params)
        boot_se = np.std(boot_params_array, axis=0)
        boot_mean = np.mean(boot_params_array, axis=0)
        
        print(f'\nBootstrap Results ({len(bootstrap_params)} successful replications)')
        print('=' * 70)
        
        boot_table = pd.DataFrame({
            'Variable': var_names_honore,
            'Point Estimate': honore_result.params,
            'Boot. Mean': boot_mean,
            'Boot. SE': boot_se,
            'Boot. t-stat': honore_result.params / boot_se,
            'CI Lower (95%)': honore_result.params - 1.96 * boot_se,
            'CI Upper (95%)': honore_result.params + 1.96 * boot_se
        })
        display(boot_table)
        
        # Check for bootstrap bias
        boot_table['Bias'] = boot_mean - honore_result.params
        print('\nBootstrap bias (Boot. Mean - Point Estimate):')
        for i, name in enumerate(var_names_honore):
            print(f'  {name}: {boot_table["Bias"].iloc[i]:.4f}')
    else:
        print(f'\nOnly {len(bootstrap_params)} successful replications -- '
              'insufficient for reliable bootstrap SEs.')
        print('Increase n_bootstrap or check data quality.')

In [ ]:
# Visualize bootstrap distributions
if honore_success and len(bootstrap_params) >= 10:
    boot_params_array = np.array(bootstrap_params)
    n_vars = len(var_names_honore)
    
    fig, axes = plt.subplots(1, n_vars, figsize=(4 * n_vars, 4))
    if n_vars == 1:
        axes = [axes]
    
    for i, (name, ax) in enumerate(zip(var_names_honore, axes)):
        ax.hist(boot_params_array[:, i], bins=15, alpha=0.7,
                edgecolor='black', color='steelblue')
        ax.axvline(honore_result.params[i], color='red', linestyle='--',
                   linewidth=2, label='Point Estimate')
        ax.axvline(boot_mean[i], color='#D55E00', linestyle=':',
                   linewidth=2, label='Boot. Mean')
        ax.set_xlabel(f'$\\beta_{{{name}}}$')
        ax.set_ylabel('Frequency')
        ax.set_title(name)
        ax.legend(fontsize=8)
        ax.grid(alpha=0.3)
    
    plt.suptitle('Bootstrap Distributions of Honor\u00e9 TLAD Coefficients',
                 fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'honore_bootstrap_distributions.png', dpi=300,
                bbox_inches='tight')
    plt.show()

*Figure: Histograms of bootstrap distributions for each coefficient from the Honor\u00e9 Trimmed LAD estimator. The red dashed line marks the point estimate from the original sample, and the orange dotted line shows the bootstrap mean. Distributions centered near the point estimate indicate low bias. The spread of each distribution reflects the sampling variability of the estimator.*

In [ ]:
# Sensitivity analysis: estimate on different subsamples
if honore_success:
    print('Sensitivity Analysis: Stability Across Subsamples')
    print('=' * 60)
    
    sensitivity_results = []
    n_subsamples = 5
    
    for s in range(n_subsamples):
        np.random.seed(s * 100 + 7)
        sub_ids_s = np.random.choice(all_ids, size=50, replace=False)
        df_s = df[df['id'].isin(sub_ids_s)].copy().reset_index(drop=True)
        
        try:
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                model_s = HonoreTrimmedEstimator(
                    endog=df_s['spending'].values,
                    exog=df_s[['income', 'wealth', 'household_size', 'credit_score']].values,
                    groups=df_s['id'].values,
                    time=df_s['time'].values,
                    censoring_point=0.0
                )
                result_s = model_s.fit(verbose=False)
                
                row = {'Subsample': s + 1, 'Converged': result_s.converged}
                for i, name in enumerate(var_names_honore):
                    row[name] = result_s.params[i]
                sensitivity_results.append(row)
                
        except Exception as e:
            row = {'Subsample': s + 1, 'Converged': f'Error'}
            for name in var_names_honore:
                row[name] = np.nan
            sensitivity_results.append(row)
    
    sens_df = pd.DataFrame(sensitivity_results)
    display(sens_df)
    
    # Compute coefficient of variation across subsamples
    print('\nCoefficient of Variation (std/|mean|) across subsamples:')
    for name in var_names_honore:
        vals = sens_df[name].dropna()
        if len(vals) > 1 and vals.mean() != 0:
            cv = vals.std() / abs(vals.mean())
            print(f'  {name}: {cv:.3f}')
    
    print('\nLower CV indicates more stable estimates across subsamples.')

### 8.4 When to Use the Honoré Estimator

**Use the Honoré estimator when:**

1. You suspect **correlated individual effects** ($\alpha_i$ correlated with $\mathbf{X}_{it}$), making the RE Tobit inconsistent.
2. You want **minimal distributional assumptions** -- the estimator does not require normality of errors or of $\alpha_i$.
3. Your panel is relatively **short** ($T$ small), where the incidental parameters problem is most severe.
4. You have a **moderate sample size** ($N$ in the hundreds, not tens of thousands).

**Prefer alternatives when:**

1. **RE assumptions are plausible**: If you believe $\alpha_i \perp \mathbf{X}_{it}$ (e.g., based on Hausman-type tests), the RE Tobit is more efficient.
2. **Large datasets**: The computational cost may be prohibitive for $N > 5{,}000$.
3. **Standard errors are critical**: If you need precise confidence intervals, the RE Tobit with analytical SEs is more practical.
4. **$T$ is large**: With large $T$, bias-corrected FE MLE approaches become viable alternatives (e.g., Hahn and Newey, 2004).

### 8.5 Limitations

1. **No standard errors**: Requires bootstrap, which multiplies the already high computational cost.
2. **No intercept**: The fixed effects absorb the constant, so marginal effects at specific covariate values require additional computation.
3. **Left-censoring only**: The current PanelBox implementation handles left-censoring at a single point.
4. **Balanced panels preferred**: Unbalanced panels work but reduce the number of usable pairs.
5. **Experimental status**: The implementation may change in future PanelBox versions.

---

## 9. Summary and Key Takeaways

### What We Learned

1. **The Incidental Parameters Problem**
   - Standard fixed effects MLE for the Tobit model is **inconsistent** with fixed $T$.
   - The number of nuisance parameters ($\alpha_i$) grows with $N$, contaminating the estimation of $\boldsymbol{\beta}$.
   - This is a fundamental problem specific to **nonlinear** models -- it does not arise in linear FE.

2. **Honoré's Semiparametric Solution**
   - Uses **pairwise differencing** ($y_{it} - y_{is}$) to eliminate $\alpha_i$.
   - **Trims** pairs where both observations are censored (uninformative).
   - Minimizes a **Least Absolute Deviations** objective on the trimmed differences.
   - Consistent for $\boldsymbol{\beta}$ as $N \to \infty$ with **no distributional assumptions**.

3. **PanelBox Implementation**
   - `HonoreTrimmedEstimator(endog, exog, groups, time, censoring_point)` -- no constant in `exog`.
   - Returns `HonoreResults` with `params`, `converged`, `n_iter`, `n_obs`, `n_entities`, `n_trimmed`.
   - **Experimental**: issues `ExperimentalWarning`, computationally intensive.

4. **Comparison with Random Effects Tobit**
   - RE Tobit assumes $\alpha_i \perp \mathbf{X}_{it}$ (strong); Honoré does not.
   - RE Tobit provides standard errors; Honoré requires bootstrap.
   - Differences between the two estimators indicate **correlated individual effects**.

5. **Practical Considerations**
   - Computational cost: $O(N \cdot T^2)$; use subsamples for large data.
   - No analytical standard errors; use **entity-level bootstrap**.
   - Non-smooth optimization: convergence depends on starting values and method.
   - Sensitivity analysis across subsamples is recommended.

### PanelBox API Reference

```python
from panelbox.models.censored import HonoreTrimmedEstimator

# Create model (NO constant in exog -- FE absorbs it)
model = HonoreTrimmedEstimator(
    endog=y,
    exog=X,              # No constant column
    groups=entity_ids,
    time=time_ids,
    censoring_point=0.0
)

# Fit (returns HonoreResults dataclass)
result = model.fit(verbose=True)

# Access results
result.params       # Coefficient vector
result.converged    # Boolean convergence indicator
result.n_iter       # Number of optimization iterations
result.n_obs        # Total observations
result.n_entities   # Number of panel entities
result.n_trimmed    # Number of trimmed pairwise differences

# Model summary
print(model.summary())
```

### References

- Honoré, B. E. (1992). Trimmed LAD and least squares estimation of truncated and censored regression models with fixed effects. *Econometrica*, 60(3), 533-565.
- Neyman, J., & Scott, E. L. (1948). Consistent estimates based on partially consistent observations. *Econometrica*, 16(1), 1-32.
- Hahn, J., & Newey, W. (2004). Jackknife and analytical bias reduction for nonlinear panel models. *Econometrica*, 72(4), 1295-1319.
- Wooldridge, J. M. (2010). *Econometric Analysis of Cross Section and Panel Data* (2nd ed.). MIT Press, Chapter 16.